In [3]:
!pip install qiskit qiskit-optimization qiskit-algorithms qiskit-nature pennylane


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.5/645.5 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import pennylane as qml
import numpy as np
import itertools

# Define board size
n = 4  # Change this for different N-Queens problems

# Define quantum device with n^2 qubits
dev = qml.device("default.qubit", wires=n**2)

# Function to generate valid N-Queens solutions
def generate_valid_solutions(n):
    """Generate valid classical solutions for N-Queens problem."""
    solutions = []
    for perm in itertools.permutations(range(n)):  # All unique row placements
        board = np.zeros((n, n), dtype=int)
        for row, col in enumerate(perm):
            board[row, col] = 1
        solutions.append(board.flatten())  # Convert board to binary array
    return solutions

# Get valid solutions for oracle marking
valid_solutions = generate_valid_solutions(n)
valid_states = ["".join(map(str, sol)) for sol in valid_solutions]

# Define Oracle for marking valid solutions
def oracle():
    """Marks valid solutions by flipping their phase."""
    for valid_state in valid_states:
        binary_state = [int(bit) for bit in valid_state]
        qml.MultiControlledX(
            wires=list(range(n**2 - 1)) + [n**2 - 1],
            control_values=binary_state[:-1]  # Control on valid state
        )

# Define the Grover Diffusion Operator
def diffusion_operator():
    """Applies the Grover diffusion operator to amplify valid solutions."""
    for i in range(n**2):
        qml.Hadamard(wires=i)
        qml.PauliX(wires=i)

    # Apply multi-controlled phase flip
    qml.MultiControlledX(
        wires=list(range(n**2 - 1)) + [n**2 - 1],
        control_values=[1] * (n**2 - 1)  # Flip if all are 1
    )

    for i in range(n**2):
        qml.PauliX(wires=i)
        qml.Hadamard(wires=i)

# Define the Grover’s Algorithm Circuit
@qml.qnode(dev)
def grover_circuit():
    # Apply Hadamard to create equal superposition
    for i in range(n**2):
        qml.Hadamard(wires=i)

    # Perform Grover Iterations
    num_iterations = int(np.floor(np.pi / 4 * np.sqrt(len(valid_states))))  # Approximate iterations
    for _ in range(num_iterations):
        oracle()  # Apply oracle
        diffusion_operator()  # Apply diffusion operator

    #  Measure probability distribution
    return qml.probs(wires=range(n**2))

# Solve the N-Queens Problem
probabilities = grover_circuit()

# Get the most probable outcome
most_probable_outcome = np.argmax(probabilities)

# Convert binary outcome to a list of positions
binary_outcome = format(most_probable_outcome, f'0{n**2}b')
positions = [int(bit) for bit in binary_outcome]

# Create the N-Queens board matrix
solution_matrix = np.array(positions).reshape(n, n)

# Print the corrected solution matrix
print("Corrected Solution Matrix (1 = Queen, 0 = Empty):")
print(solution_matrix)


Corrected Solution Matrix (1 = Queen, 0 = Empty):
[[0 1 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]]


In [ ]:
import pennylane as qml
import numpy as np

# Define board size
n = 4

# Define quantum device with n^2 qubits
dev = qml.device("default.qubit", wires=n**2)

# Define Oracle for marking valid solutions
def oracle():
    """Marks valid solutions by flipping their phase."""
    qml.PauliZ(wires=0)  # Example placeholder oracle operation

# Define the Grover Diffusion Operator
def diffusion_operator():
    """Applies the Grover diffusion operator to amplify valid solutions."""
    for i in range(n**2):
        qml.Hadamard(wires=i)
        qml.PauliX(wires=i)

    # Apply a multi-controlled phase flip
    qml.Hadamard(wires=n**2 - 1)
    qml.PauliX(wires=n**2 - 1)
    qml.CZ(wires=[n**2 - 2, n**2 - 1])  # Example controlled phase flip
    qml.PauliX(wires=n**2 - 1)
    qml.Hadamard(wires=n**2 - 1)

    for i in range(n**2):
        qml.PauliX(wires=i)
        qml.Hadamard(wires=i)

# Define the Grover’s Algorithm Circuit
@qml.qnode(dev)
def grover_circuit():
    """Implements Grover's algorithm to find N-Queens solutions."""
    # Step 1: Apply Hadamard to create equal superposition
    for i in range(n**2):
        qml.Hadamard(wires=i)

    # Step 2: Perform Grover Iterations (1 iteration for visualization)
    oracle()  # Apply oracle
    diffusion_operator()  # Apply diffusion operator

    return qml.probs(wires=range(n**2))

# Draw the quantum circuit
circuit_diagram = qml.draw(grover_circuit)()
print(circuit_diagram)


 0: ──H──Z──H──X──X──H─────────────┤ ╭Probs
 1: ──H──H──X──X──H────────────────┤ ├Probs
 2: ──H──H──X──X──H────────────────┤ ├Probs
 3: ──H──H──X──X──H────────────────┤ ├Probs
 4: ──H──H──X──X──H────────────────┤ ├Probs
 5: ──H──H──X──X──H────────────────┤ ├Probs
 6: ──H──H──X──X──H────────────────┤ ├Probs
 7: ──H──H──X──X──H────────────────┤ ├Probs
 8: ──H──H──X──X──H────────────────┤ ├Probs
 9: ──H──H──X──X──H────────────────┤ ├Probs
10: ──H──H──X──X──H────────────────┤ ├Probs
11: ──H──H──X──X──H────────────────┤ ├Probs
12: ──H──H──X──X──H────────────────┤ ├Probs
13: ──H──H──X──X──H────────────────┤ ├Probs
14: ──H──H──X───────╭●──X──H───────┤ ├Probs
15: ──H──H──X──H──X─╰Z──X──H──X──H─┤ ╰Probs
